# Configuracion del directorio

In [7]:
import os

data_dirname = os.path.join(os.getcwd(), "data")
QUERY = "Reforma al poder judicial"

# Recoleccion de datos por paginas web

In [ ]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search


def search_pages_google(query: str):
    return list(search(query, num_results=10))


def extract_page_content(url: str):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        main_content = (
            soup.find("main")
            or soup.find("article")
            or soup.find("div", class_="content")
        )
        if main_content:
            return main_content.get_text(strip=True, separator="\n")
        else:
            return soup.get_text(strip=True, separator="\n")
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
        return None


def search_in_pages():
    query = QUERY
    urls = search_pages_google(query)
    results = []
    for url in urls:
        content = extract_page_content(url)
        if content:
            results.append(content)

    for result in results:
        print(result)


search_in_pages()

# Recolección de datos por pdf
Usando googlesearch en conjunto con requests, se extraen pdfs

In [4]:
import os
import requests
from googlesearch import search

pdf_dirname = os.path.join(data_dirname, "pdfs")


def download_pdf(url: str) -> None:
    try:
        response = requests.get(url)
        response.raise_for_status()
        filename = url.split("/")[-1]
        if not filename.endswith(".pdf"):
            filename += ".pdf"
        os.makedirs(pdf_dirname, exist_ok=True)
        filepath = os.path.join(pdf_dirname, filename)
        with open(filepath, "wb") as f:
            f.write(response.content)
        print(f"Descargado: {filepath}")
    except Exception as e:
        print(f"Error al descargar {url}: {e}")


def search_pdfs():
    query = "Reforma Judicial Mexico filetype:pdf after:2024-01-01"
    results = search(query, num_results=50)
    for url in results:
        if url.endswith(".pdf"):
            download_pdf(url)


search_pdfs()

Descargado: c:\Users\eldav\ia-2-1\reforma\data\pdfs\BoletinFNo125_5.pdf
Descargado: c:\Users\eldav\ia-2-1\reforma\data\pdfs\REFORMA_AL_PODER_JUDICIAL_CS.pdf
Error al descargar https://www.diputados.gob.mx/sedia/sia/spi/SAPI-ASS-11-24.pdf: HTTPSConnectionPool(host='www.diputados.gob.mx', port=443): Max retries exceeded with url: /sedia/sia/spi/SAPI-ASS-11-24.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))
Error al descargar https://escuelajudicial.cjf.gob.mx/MicroSitio/Doctos/Minuta_Reforma_Poder_Judicial.pdf: HTTPSConnectionPool(host='escuelajudicial.cjf.gob.mx', port=443): Max retries exceeded with url: /MicroSitio/Doctos/Minuta_Reforma_Poder_Judicial.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))
Descargado: c:\Users\eldav\ia-2-1\reforma\data\pd

In [5]:
from PyPDF2 import PdfReader


def extract_text_from_pdf(filepath: str) -> str:
    try:
        text = ""
        with open(filepath, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                text += page.extract_text()
                text += "\n"
        return text
    except Exception as e:
        return ""


def clean_text(text: str) -> str:
    lines = text.splitlines()
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    return "\n".join(cleaned_lines)


def create_pdfs_txt():
    text = ""

    for filename in os.listdir(pdf_dirname):
        filepath = os.path.join(pdf_dirname, filename)
        print(f"Extrayendo texto de {filepath}")
        text += extract_text_from_pdf(filepath)
    text = clean_text(text)
    output_file = os.path.join(data_dirname, "pdfs.txt")
    with open(output_file, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)


create_pdfs_txt()

Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\05092024m-columnaspoliticas.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\100420241844096360.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\20221355_StatementMEX-LopezObrador_Spanish.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\20240902_PublicacionMCV_Reforma_Poder_Judicial-v2.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\21.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\21369.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\22.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\240910_PPT_PJ__CS.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\24102024m-nacional.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf
Extrayendo texto de c:\Users\eldav\ia-2-1\reforma\data\pdfs\Analisis-de-iniciativa-para